# Fuzzy C means Clustering

In [ ]:

#own implementation of fcmeans
import numpy as np
import pandas as pd
import random
import math
import operator
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from openpyxl import load_workbook
# Parameters for fcmeans clustering
MAX_ITER = 2 #maximum number of iterations
m = 2 # fuzziness coeff

def fuzzyCMeansClustering(data,c): 
    membership_mat = uinitial(data,c)
    for curr in range(MAX_ITER):
        cluster_labels = getClusters(data,c,membership_mat)      
        cluster_centers = centroids(data,c,membership_mat)
        membership_mat = uij(data,c,membership_mat, cluster_centers)
        labels = cluster_labels
    return labels, cluster_centers

def uinitial(data,c): # initializing the membership matrix
    membership_mat = []
    n = len(data)
    for i in range(n):
        random_num_list = [random.random() for i in range(c)] #generate list of c random numbers
        summation = sum(random_num_list) 
        temp = [x/summation for x in random_num_list]  
        
        f = temp.index(max(temp)) # get index of max value
        for j in range(0,len(temp)):
            if(j == f):
                temp[j] = 1
            else:
                temp[j] = 0
        
        membership_mat.append(temp)
    return membership_mat 

def getClusters(data,c,membership_mat): # getting the cluster labels
    cluster_labels = list()
    n=len(data)
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
        cluster_labels.append(idx)
    return cluster_labels

def centroids(data,c,membership_mat): # calculating the cluster center
    cluster_mem_val = list(zip(*membership_mat)) #unpacking membership matrix
    cluster_centers = []
    n= len(data)
    for j in range(c):
        u = list(cluster_mem_val[j]) # gets each row as a list
        u_raised = [p ** m for p in u] # calculates uij raised to m
        denominator = sum(u_raised) # ∑(uij ** m)
        temp_num = []
        for i in range(n):
            x = list(data.iloc[i]) 
            prod = [u_raised[i] * val for val in x] 
            temp_num.append(prod)
        numerator = map(sum, list(zip(*temp_num))) # numerator = ∑(uij*xi)
        cj = [z/(denominator+1) for z in numerator] # cj =  ∑(uij*xi)/∑(uij ** m) 
        cluster_centers.append(cj)
    return cluster_centers

def uij(data,c,membership_mat, centroids): # Updating the membership value
    p = float(2/(m-1)) 
    n=len(data)
    for i in range(n):
        x = list(data.iloc[i])
        distances = [np.linalg.norm(np.array(list(map(operator.sub, x, centroids[j])))) for j in range(c)] #  calculates ∥xi - cj∥
        for j in range(c):
            den = sum([math.pow(float(distances[j]/distances[k]), p) for k in range(c)]) # Σ(∥xi - cj∥/∥xi - ck∥)**(2/m-1)
            membership_mat[i][j] = float(1/den)  # update uij
    return membership_mat


silhv= np.zeros((56,9))
dbind=np.zeros((56,9))
for i in range(1,57):
    fname = "/Users/durbasatpathi/Desktop/datamining/data/"+ str(i)+".csv"
    data = pd.read_csv(fname,delimiter=',',header=None)
    data.drop(data.columns[len(data.columns)-1],axis=1,inplace=True)
    print(i)
    for j in range(2,11):
        
        fcmeans_labels = fuzzyCMeansClustering(data,j)[0]
        silhv[i-1,j-2]=silhouette_score(data,fcmeans_labels,metric='euclidean'); 
        dbind[i-1,j-2]=davies_bouldin_score(data,fcmeans_labels)
        print(j,silhv[i-1,j-2],dbind[i-1,j-2])
        
path = r"/Users/durbasatpathi/Desktop/2019A7PS0972H_silhv.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book
df=pd.DataFrame(silhv)
df.to_excel(writer, sheet_name='fcmeans')
#writer.save()
path = r"/Users/durbasatpathi/Desktop/2019A7PS0972H_db.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book
df=pd.DataFrame(dbind)
df.to_excel(writer, sheet_name='fcmeans')
#writer.save()

1
2 0.5583505739204777 0.7654875113134364
3 0.3355548366587066 0.8366598381816671
4 0.31330352476574996 0.77063787664751
5 0.25684929802286516 1.394151325399066
6 0.2548294295497653 1.3110484272213008
7 0.16385982792905096 1.1231703793084644
8 0.10707677974970185 1.1561185577696316
9 0.22003361145686787 1.014323161890236
10 0.2274060115946619 1.027646737241764
2
2 0.5872755976986066 0.7596239715380833
3 0.5654535982788244 1.3505138511436101
4 0.18142788052933473 0.9165787498030978
5 0.2585325137110851 1.315169729546096
6 0.1394398320851392 1.2137674182262346
7 0.2243097948363508 1.3514670367629493
8 0.202534624878332 1.2489811441890157
9 0.18033335355399874 1.07277158632773
10 0.17089051496748917 1.3753755823310105
3
2 0.6215272292155809 0.8160377173590146
3 0.465704745420825 1.2441497003799398
4 0.23785734611980813 0.9176360903959647
5 0.27777755761671813 1.543292794431703
6 0.30181423174289396 1.279155229684256
7 0.30110749769467327 1.1790981989615439
8 0.20929850728404623 1.32810619

6 0.17467979964917993 1.35638195370222
7 0.1790351065735006 1.0768413954753764
8 0.10338103424019922 1.2425306574440098
9 0.14891163047709366 1.283490383632894
10 0.11351464991335111 1.4519922225607316
24
2 0.510857538892944 0.7950396347517366
3 0.19823127987891026 0.8691392240487564
4 0.15853777527225094 0.8769837754581153
5 0.18362281596541097 1.2141376220031947
6 0.17932330406033545 1.0118041716915018
7 0.19766870950782553 1.3080993905501965
8 0.13966043455257415 1.159387266521865
9 0.11392843484657922 1.4833982391956346
10 0.157124791113719 1.5417439369751456
25
2 0.5307565543565871 1.5399648313312972
3 0.30334521487323984 1.0707633570457096
4 0.19686014681004457 1.6136018497904292
5 0.2013403012785652 1.1384756631082473
6 0.24644680303424468 1.5791071243907107
7 0.1696838719762542 1.3613630419916924
8 0.20084085861888976 1.3637481605300537
9 0.13973200164849403 1.6303832313105509
10 0.1368682883341148 1.1863890310879908
26
2 0.5402780816559475 0.8824978553413483
3 0.31946201094351

In [10]:
#library implemetation of fcmeans
from fcmeans import FCM
import numpy as np
from sklearn import metrics
for i in range(1,57):
    print(i)
    fname='/Users/durbasatpathi/Desktop/datamining/data/'+str(i)+'.csv'
    data=np.genfromtxt(fname,delimiter=',');
    for j in range(2,11):
        fcm = FCM(n_clusters=j)
        fcm.fit(data[:,0:-1])
        fcm_labels=fcm.predict(data[:,0:-1])
        a=metrics.silhouette_score(data[:,0:-1],fcm_labels, metric='euclidean')
        b=metrics.davies_bouldin_score(data[:,0:-1], fcm_labels)
        print(j,a,b)

1
2 0.6516667938820518 0.7791321097824433
3 0.6368228410706144 0.5015883474334916
4 0.5433478849623528 0.591239252701911
5 0.570011650439244 0.5458216844466491
6 0.4520422094175051 0.6275708625417368
7 0.4204938638953183 0.7203553270107987
8 0.438105968277005 0.7041665380392399
9 0.420597209535359 0.7065427081464744
10 0.41349214860667244 0.7692158465602308
2
2 0.6648551882775535 0.783267430411337
3 0.6232999133149253 0.8050756950218219
4 0.5529596848702256 0.8885848245108997
5 0.4468387345434964 0.9348740224240885
6 0.47823286861094244 0.6892956952485375
7 0.4266733778661361 0.8285277572803269
8 0.42057781297778396 0.8607788128038443
9 0.45652439864049477 0.6621628919730422
10 0.4074743256560776 0.8006306078285356
3
2 0.7177377911925032 0.7995628756405131
3 0.7039131255210155 0.6958654525167898
4 0.6565746638338869 0.7273573252477794
5 0.571768030031332 0.7781356460209247
6 0.5495683220757592 0.8675251333783095
7 0.47137185370144674 0.8847069260917747
8 0.4662390597439971 0.8372675445

7 0.4652167841402085 0.7135215342974933
8 0.4320549502798435 0.7557552108615283
9 0.3738587096600215 0.82564095041624
10 0.37512979042125777 0.8133714100236619
24
2 0.6144547796893041 0.6901976183346035
3 0.563929823582038 0.6602692534600298
4 0.5144113277156508 0.6364553978660347
5 0.5315999904470176 0.7647138216062499
6 0.4443616483463823 0.8808851798609476
7 0.3889122326198335 1.0017439130721537
8 0.4059822968957329 0.8773600774363894
9 0.3561855814959481 0.9190802507885558
10 0.3240943848924276 1.004951306827007
25
2 0.7321663439727735 0.9972402071774649
3 0.6240312155923295 0.49835220788800755
4 0.5641251155915321 0.5450982825969344
5 0.5005400733640373 0.6106863572231902
6 0.4853777344219791 0.7250605318889854
7 0.4995823392303941 0.6953158633338503
8 0.43129923181707175 0.7511796794484441
9 0.4227941793223074 0.758148796599161
10 0.4172635550649645 0.8054270920038702
26
2 0.9008088392130524 0.2391750313691371
3 0.6477406047839348 0.5971291274060397
4 0.5885460528247958 0.6555513

3 0.7456814259839836 0.24791916125942706
4 0.6277647080319485 0.3376575816524666
5 0.5372942182525215 0.42166338586196855
6 0.5106021048722597 0.5291793840028737
7 0.5115898306351668 0.4566984630999887
8 0.47630924650974193 0.5389949869263926
9 0.4594265522973637 0.5773113509876766
10 0.4447918273571311 0.39738067316228187
47
2 0.8324989020728265 0.9262745100347658
3 0.7791503820416688 0.7679457388617723
4 0.6892196641470283 0.8246782021040677
5 0.6869729445957242 0.8250744409708396
6 0.6061939079400851 0.8487405865859307
7 0.5788478292307279 0.83946291545447
8 0.5506083496508933 0.8607369423328917
9 0.5364034253669393 0.8489209414420975
10 0.5329277099157644 0.9160675473935214
48
2 0.973491388764493 0.40076882089634264
3 0.6678454417931411 0.6649888932475189
4 0.558522398171457 0.7988563715556499
5 0.5240090129498257 0.5277556604462987
6 0.50069983352548 0.6436122030978005
7 0.4807827335532164 0.6590451275374546
8 0.4887038995628343 0.6022728916427158
9 0.46599460238239937 0.691918795